<a href="https://colab.research.google.com/github/Ademola-Olorunnisola/TB-Estimator/blob/main/Nonlinear_Observability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Symbolic nonlinear observability

In [72]:
import numpy as np
import sympy as sp
from IPython.display import display

In [73]:
# Import functions directly from github
# Important: note that we use raw.githubusercontent.com, not github.com

import requests
url = 'https://raw.githubusercontent.com/florisvb/Nonlinear_and_Data_Driven_Estimation/main/Utility/symbolic_derivatives.py'
r = requests.get(url)

# Store the file to the colab working directory
with open('symbolic_derivatives.py', 'w') as f:
    f.write(r.text)

# import the function we want from that file
import symbolic_derivatives

# Define states and dynamics in control affine form

In [74]:
# Define states
S, V, E, I, T, R, beta, eps, gamma, sigma, phi, rho = sp.symbols(['S', 'V', 'E', 'I', 'T', 'R', 'beta', 'eps', 'gamma', 'sigma', 'phi', 'rho'])
x = [S, V, E, I, T, R, beta, eps, gamma, sigma, phi, rho]

# Parameters
Lambda, mu, delta = sp.symbols(['Lambda', 'mu', 'delta'])
lam = beta * I / N

f_0 = sp.Matrix([
    Lambda - lam*S - mu*S,
    -sigma*lam*V - mu*V,
    lam*S + sigma*lam*V + (phi*R*I)/N - (eps+mu)*E,
    eps*E + rho*T - (mu + delta)*I,
     - (gamma + rho + mu)*T,
    gamma*T - mu*R - (phi*R*I)/N,
    0,
    0,
    0,
    0,
    0,
    0
])

f_1 = sp.Matrix([
    -S,
    S,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0
])

f_2 = sp.Matrix([
    0,
    0,
    0,
    -I,
    I,
    0,
    0,
    0,
    0,
    0,
    0,
    0
])

In [75]:
x0 = {
    S: 390000,
    V: 10000,
    E: 40000,
    I: 25000,
    T: 30000,
    R: 5000,
    beta: 0.010,
    eps: 1 / 365,
    gamma: 1 / 180,
    sigma: 0.50,
    phi: 0.03,
    rho: 0.21 * (1 / 180),
    Lambda: 10.958904109589042 ,
    mu: 8 / 1000 / 365,
    delta: 1 / 1000 / 365 ,
    N: 500000
}

# Define measurements

In [76]:
# Measurement function
N = sp.symbols('N')  # Total population (constant)

h = sp.Matrix([I, V, R, E, T])

# Calculate each term in G

$G = [h, L_{f_0}h, L_{f1}h]$

In [77]:
# Calculate each term in G
# G = [h, L_{f_0}h, L_{f_1}h, L_{f_2}h]

# Take the derivative of h with respect to x along the vector f_0
L_f0_h = symbolic_derivatives.directional_derivative(h, x, f_0)
display(L_f0_h)

print('')

# Take the derivative of h with respect to x along the vector f_1
L_f1_h = symbolic_derivatives.directional_derivative(h, x, f_1)
display(L_f1_h)

print('')

# Take the derivative of h with respect to x along the vector f_2
L_f2_h = symbolic_derivatives.directional_derivative(h, x, f_2)
display(L_f2_h)

Matrix([
[                           E*eps - I*(delta + mu) + T*rho],
[                                 -I*V*beta*sigma/N - V*mu],
[                              -I*R*phi/N - R*mu + T*gamma],
[-E*(eps + mu) + I*R*phi/N + I*S*beta/N + I*V*beta*sigma/N],
[                                    T*(-gamma - mu - rho)]])

Matrix([
[0],
[S],
[0],
[0],
[0]])

Matrix([
[-I],
[ 0],
[ 0],
[ 0],
[ I]])

# Assemble G, take Jacobian

In [78]:
# Assemble G, take Jacobian
G = sp.Matrix.vstack(h, L_f0_h, L_f1_h, L_f2_h)
display(G)

Matrix([
[                                                        I],
[                                                        V],
[                                                        R],
[                                                        E],
[                                                        T],
[                           E*eps - I*(delta + mu) + T*rho],
[                                 -I*V*beta*sigma/N - V*mu],
[                              -I*R*phi/N - R*mu + T*gamma],
[-E*(eps + mu) + I*R*phi/N + I*S*beta/N + I*V*beta*sigma/N],
[                                    T*(-gamma - mu - rho)],
[                                                        0],
[                                                        S],
[                                                        0],
[                                                        0],
[                                                        0],
[                                                       -I],
[              

In [79]:
# Jacobian of G with respect to states
display(G.jacobian(x))

Matrix([
[       0,                    0,         0,                                   1,                 0,             0,                   0,  0,  0,           0,      0,  0],
[       0,                    1,         0,                                   0,                 0,             0,                   0,  0,  0,           0,      0,  0],
[       0,                    0,         0,                                   0,                 0,             1,                   0,  0,  0,           0,      0,  0],
[       0,                    0,         1,                                   0,                 0,             0,                   0,  0,  0,           0,      0,  0],
[       0,                    0,         0,                                   0,                 1,             0,                   0,  0,  0,           0,      0,  0],
[       0,                    0,       eps,                         -delta - mu,               rho,             0,                   0,  E,  

# Check the rank of G for a given operating point, $x_0$

In [80]:
display(G.jacobian(x).subs(x0))

print('')
print('Rank of G:')
G.jacobian(x).subs(x0).rank()

Matrix([
[     0,                     0,                    0,                    1,                   0,                    0,       0,      0,      0,    0,    0,      0],
[     0,                     1,                    0,                    0,                   0,                    0,       0,      0,      0,    0,    0,      0],
[     0,                     0,                    0,                    0,                   0,                    1,       0,      0,      0,    0,    0,      0],
[     0,                     0,                    1,                    0,                   0,                    0,       0,      0,      0,    0,    0,      0],
[     0,                     0,                    0,                    0,                   1,                    0,       0,      0,      0,    0,    0,      0],
[     0,                     0,  0.00273972602739726, -2.46575342465753e-5, 0.00116666666666667,                    0,       0,  40000,      0,    0,    0,  30000],
[


Rank of G:


11

# Shortcut function to get G:

### First derivatives

In [81]:
# First derivatives
G1 = symbolic_derivatives.get_bigO(h, x, [f_0, f_1, f_2])

# Second derivatives
G2 = symbolic_derivatives.get_bigO(sp.Matrix.vstack(*G1), x, [f_0, f_1, f_2])

# Both first and second derivatives
G = sp.Matrix.vstack(*G1, *G2)
display(G)

Matrix([
[                                                                                                                                                                                                                                                                            I],
[                                                                                                                                                                                                                                                                            V],
[                                                                                                                                                                                                                                                                            R],
[                                                                                                                                                                           

# Exercises:

1. Is the system observable with no controls (i.e. $u=0$)?
2. Is the system observable with control? (i.e. $u\neq0$)?
3. How many derivatives are needed, 1 or 2?
3. Apply the symbolic approach to the planar drone example with the measurements below. What is necessary in order for $z$ to be observable?